## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [37]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [38]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
df



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,0.367,0.2,1.1,0.217,1.3,3.0,0.423,0.396,0.5,0.6,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,0.339,1.4,4.4,0.325,0.9,2.5,0.363,0.442,1.2,1.4,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,0.529,0.0,0.1,0.100,3.2,5.8,0.539,0.530,2.4,3.2,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [39]:
# answer goes here
df = df[df['GS'] >= 20]
df = df[df['MP'] >= 10]
df




,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,0.414,1.5,4.3,0.344,3.4,7.6,0.454,0.477,2.8,3.2,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,0.408,1.6,4.8,0.344,2.6,5.6,0.464,0.488,5.3,6.3,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,0.452,1.9,4.7,0.402,2.3,4.6,0.504,0.554,1.5,1.8,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [40]:
# answer goes here
X = df[['PTS','TRB','AST','STL','BLK','TOV']]
X




,PTS,TRB,AST,STL,BLK,TOV
3,13.9,9.5,1.6,1.5,1.0,1.7
4,8.9,7.3,2.2,0.9,0.8,1.5
7,21.3,9.2,2.4,0.5,1.3,1.8
10,10.9,8.4,1.4,0.5,1.5,1.3
12,9.4,7.5,1.3,0.8,0.4,0.9
...,...,...,...,...,...,...
2122,14.1,2.9,5.8,0.9,0.2,2.3
2125,15.3,2.5,2.5,0.9,0.3,1.6
2126,11.7,6.4,1.4,0.7,1.0,0.8
2138,15.1,9.0,1.8,1.5,0.5,1.9


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [45]:
# answer goes here
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
X_scale



,PTS,TRB,AST,STL,BLK,TOV
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854
...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852


Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [46]:
# answer goes here
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
X_scale['cluster'] = kmeans.fit_predict(X_scale)
X_scale


,PTS,TRB,AST,STL,BLK,TOV,cluster
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511,2
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831,2
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681,2
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172,2
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854,1
...,...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535,0
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660,1
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025,2
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852,2


In [47]:
centers = pd.DataFrame(kmeans.cluster_centers_,columns=X.columns)
centers

,PTS,TRB,AST,STL,BLK,TOV
0,1.075975,-0.001260,1.454813,1.042428,-0.213796,1.353680
1,-0.468976,-0.517071,-0.396048,-0.279474,-0.439358,-0.539769
2,0.086332,1.269641,-0.467217,-0.345349,1.289195,-0.014648


In [49]:
center_inverse = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
center_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765
1,10.505405,3.949775,2.063739,0.804730,0.374324,1.235135
2,13.735359,8.605525,1.914917,0.777348,1.230387,1.668508


First cluster has a lot of global maximum values and the second cluster has none. It shows that those 6 stats are correlated in that players with high points scored also assist and steal well. 

Experiment with different values of K. Do any further interesting archetypes come out?

In [51]:
# answer goes here
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
kmeans = KMeans(n_clusters=6)
X_scale['cluster'] = kmeans.fit_predict(X_scale)
centers = pd.DataFrame(kmeans.cluster_centers_,columns=X.columns)
centers

,PTS,TRB,AST,STL,BLK,TOV
0,-0.016071,-0.492485,-0.029652,0.268715,-0.522728,-0.143640
1,0.766542,1.980317,-0.268169,0.024664,2.300327,0.547921
2,1.814832,1.005109,2.250192,1.796358,0.403791,2.548445
3,0.913890,-0.254062,1.292131,0.849616,-0.401386,1.082598
4,-0.949944,-0.694300,-0.710955,-0.766978,-0.478430,-0.938876
5,-0.303676,0.738885,-0.585835,-0.581055,0.661203,-0.362721


In [52]:
center_inverse = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
center_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,13.139732,4.013839,2.829911,1.032589,0.333036,1.562054
1,17.691803,10.457377,2.331148,0.931148,1.731148,2.132787
2,23.789189,7.916216,7.597297,1.667568,0.791892,3.783784
3,18.548855,4.635115,5.593893,1.274046,0.393130,2.574046
4,7.707853,3.487958,1.405236,0.602094,0.354974,0.905759
5,11.466875,7.222500,1.666875,0.679375,0.919375,1.381250


shows similar trends comparing to 3 clusters